In [1]:
import numpy as np

In [9]:
eps = 1e-5
n = 10000
x = np.random.beta(10, 10, size = n)
x_star = np.max(x)
print(x_star)

def offer(i, p):
    global regret, count
    is_accepted = x[i] > p
    regret += x_star - is_accepted * x[i]
    count += 1
    return is_accepted

0.8476627200369931


### our alg

In [10]:
regret = 0
count = 0

I = np.arange(n).tolist()
beta = n * [1]
a = n * [0]
b = n * [1]
s = 0
t = n-1

while beta[s] > eps:
    if offer(I[s], a[t]):
        p_start, p_end = a[t], b[t]
        while offer(I[s], p_end):
            p_start, p_end = b[t], b[t-1]
            t = t - 1
            I.pop()
            beta.pop()
            a.pop()
            b.pop()

        beta.append(np.minimum(0.5, beta[s]) * (p_end - p_start))
        p = p_start + beta[t + 1]
        while offer(I[s], p):
            p = p + beta[t + 1]
            if p >= p_end:
                break
                
        I.append(I[s])
        a.append(p - beta[t + 1])
        b.append(p)
        s = s + 1
        t = t + 1
    else:
        s = s + 1

In [11]:
print(s)
print(x_star)
print(x[I[s]])
print(a[s], b[s])
print("regret = ", regret)
print("count = ", count)

10004
0.8476627200369931
0.8476627200369931
0.8476626873016357 0.8476627469062805
regret =  8492.284012722776
count =  10177


### independent incremental search

In [12]:
regret = 0
count = 0

a = np.zeros(n)
b = np.ones(n)
beta = 0.5 * np.ones(n)

while True:
    ucb_idx = np.argmax(b)
    if b[ucb_idx] - a[ucb_idx] < eps:
        break
        
    p = a[ucb_idx] + beta[ucb_idx]
    while offer(ucb_idx, p):
        p = p + beta[ucb_idx]
    a[ucb_idx] = p - beta[ucb_idx]
    b[ucb_idx] = p
    beta[ucb_idx] = beta[ucb_idx] ** 2

In [13]:
ucb_idx = np.argmax(b)

print(ucb_idx)
print(x_star)
print(x[ucb_idx])
print("regret = ", regret)
print("count = ", count)

6493
0.8476627200369931
0.8476627200369931
regret =  14151.638208463604
count =  48101


### minimum number of queries

In [14]:
regret = 0
count = 0

I = np.arange(n).tolist()
a = n * [0]
b = n * [1]
s = 0
t = n-1

while True:
    if b[s] - a[s] < eps or s == t:
        break
    a_max = np.max(a[s:(t+1)])
    b_min = np.min(b[t])
    p = (a_max + b_min)/2
    p_prev = a[s]
    while offer(I[s], p):
        t = t - 1
        I.pop()
        a.pop()
        b.pop()
        p_prev = p
        p = np.min(b[t])

    I.append(I[s])
    a.append(p_prev)
    b.append(p)
    s = s + 1
    t = t + 1

In [15]:
print(s)
print(x_star)
print(x[I[s]])
print(a[s], b[s])
print("regret = ", regret)
print("count = ", count)

10005
0.8476627200369931
0.8476627200369931
0.84765625 0.859375
regret =  8663.687415480332
count =  20004
